In [52]:
import pandas as pd
from seml import get_results

In [ ]:
df = get_results(
    'dp_timeseries_dp_train_standard_eval_label_noise',
    [
        'config.seed',
        'config.estimator_name',
        'config.neighboring_relation.future_target_sensitivity',
        'config.dp_optimizer_kwargs.future_target_noise_multiplier',
        'config.dp_optimizer_kwargs.max_grad_norm',
        'config.dp_optimizer_kwargs.noise_multiplier',
        'config.estimator_kwargs.relative_context_length',
        'config.dp_accountant_kwargs.budget_epsilon',
        'result.metrics_test.mean_wQuantileLoss',
        'result.metrics_test.MASE',
    ],
    to_data_frame=True,
    filter_dict={
        'config.dp_optimizer_kwargs.future_target_noise_multiplier': {'$in': [0.0, 1.0, 2.0, 5.0]},
        'config.dataset_kwargs.dataset_name': 'solar_10_minutes',
        'config.estimator_kwargs.trainer_kwargs.max_epochs': 16000,
        'config.top_level_mode': 'sampling_without_replacement',
        'config.instances_per_sequence': 1,
        'config.estimator_kwargs.batch_size': {'$in': [128]},
        'config.dp_optimizer_kwargs.max_grad_norm': {'$in': [0.001, 0.0001]},
        'config.dp_optimizer_kwargs.noise_multiplier': {'$in': [2.0, 4.0]},
        'config.estimator_kwargs.relative_context_length': {'$in': [2, 4]},
        'config.dp_accountant_kwargs.budget_delta': 1e-7,
        'config.estimator_kwargs.lags_seq': {'$in': [None, [
            1, 2, 3, 4, 5, 6, 7, 23, 24, 25, 47, 48, 49, 71, 72, 73,
            95, 96, 97, 119, 120, 121, 143, 144, 145, 167, 168, 169]]},
        'config.tight_privacy_loss': True
    }
)

In [54]:
columns = df.columns[1:]

df = df.rename(columns={
    c: c.split('.')[-1]
    for c in columns
})

In [ ]:
df

In [56]:
def create_table(df: pd.DataFrame,
                 relative_context_length: int,
                 max_grad_norm: float,
                 noise_multiplier: float,
                 budget_epsilon: float,
                 future_target_sensitivity: float,
                 errors: bool = False
) -> pd.DataFrame:

    df = df.copy()

    df = df[
        (df['relative_context_length'] == relative_context_length)
        & (df['max_grad_norm'] == max_grad_norm)
        & (df['noise_multiplier'] == noise_multiplier)
        & (df['budget_epsilon'] == budget_epsilon)
        & (df['future_target_sensitivity'] == future_target_sensitivity)
    ]

    print(len(df))

    # Average over random seeds
    if not errors:
        df = df.groupby(
            ['future_target_noise_multiplier', 'estimator_name']
        ).mean().reset_index()
    else:
        df = df.groupby(
            ['future_target_noise_multiplier', 'estimator_name']
        ).std().reset_index()

    print(df.columns)

    df = df[['estimator_name', 'future_target_noise_multiplier', 'MASE', 'mean_wQuantileLoss']]

    df = df.sort_values(by=['estimator_name', 'future_target_noise_multiplier'])

    return df

### Relative context length 4, noise multiplier 4

In [ ]:
df_mean = create_table(df, 4, 0.0001, 4.0, budget_epsilon=0.5, future_target_sensitivity=0.01)

df_mean

In [ ]:
df_std = create_table(df, 4, 0.0001, 4.0, errors=True, budget_epsilon=0.5, future_target_sensitivity=0.01)

df_std

In [ ]:
i = 0
names = ['DLinear', 'DeepAR', 'iTransf.', 'SimpleFF']

acc = f'{names[0]} & ? &'

for i, (mean, std) in enumerate(zip(df_mean['mean_wQuantileLoss'], df_std['mean_wQuantileLoss'])):

    if (i > 0) and ((i % 4)  == 0):
        acc += ' \\\\'
        print(acc)
        acc = f'{names[i // 4]} & ? &'
    acc += ' $' + f'{mean:.3f}' + '$ '
    acc += '\\tiny{$\\pm ' + f'{std:.3f}' + '$} &'

acc += ' \\\\ '
print(acc)